In [1]:
#cell 1 imports
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import RMSprop
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
#paths config
base_dir = r"D:/School/Project/Machine_Learning/Datasets"
train_dir = os.path.join(base_dir, "Train")
val_dir = os.path.join(base_dir, "Validation")
output_dir = os.path.join(base_dir, "Models", "Focused_Classes_Gravel_Murram")
os.makedirs(output_dir, exist_ok=True)

IMG_HEIGHT, IMG_WIDTH = 120, 160
BATCH_SIZE = 8
EPOCHS = 20
target_focus = ['Gravel_Stony', 'Murram']

In [3]:
model = load_model(os.path.join(base_dir, "retrained_model5.h5"))
model.trainable = True

In [4]:
# Freeze all layers first
for layer in model.layers:
    layer.trainable = False

# Unfreeze last 20 layers
for layer in model.layers[-20:]:
    layer.trainable = True


In [5]:
# Cell: Set focus
target_focus = ['Gravel_Stony', 'Murram']

# Cell: Define custom iterator
from tensorflow.keras.preprocessing.image import DirectoryIterator

class FocusedDirectoryIterator(DirectoryIterator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_indices_rev = {v: k for k, v in self.class_indices.items()}

    def _get_batches_of_transformed_samples(self, index_array):
        batch_x, batch_y = super()._get_batches_of_transformed_samples(index_array)
        for i, label in enumerate(batch_y):
            class_index = np.argmax(label)
            class_name = self.class_indices_rev[class_index]
            if class_name in target_focus:
                img = batch_x[i]
                img = tf.image.random_contrast(img, 0.6, 1.4)
                img = tf.image.random_saturation(img, 0.6, 1.4)
                img = tf.image.random_brightness(img, max_delta=0.2)
                img = tf.image.random_hue(img, max_delta=0.08)
                img = tf.clip_by_value(img, 0.0, 1.0)
                batch_x[i] = img
        return batch_x, batch_y

# Cell: Define generators
train_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.6, 1.4],
    fill_mode='nearest'
)

val_aug = ImageDataGenerator(rescale=1./255)

train_data = FocusedDirectoryIterator(
    directory=train_dir,
    image_data_generator=train_aug,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_data = val_aug.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

class_names = sorted(train_data.class_indices)
print(f"All classes: {class_names}")


Found 14673 images belonging to 8 classes.
Found 625 images belonging to 8 classes.
All classes: ['Grass_Paths', 'Gravel_Stony', 'Murram', 'Pavements', 'Potholes', 'Stairs', 'Tarmac', 'Tiles']


In [6]:
labels = train_data.classes
class_weights_raw = compute_class_weight(class_weight='balanced', classes=np.unique(labels), y=labels)
class_weights = dict(enumerate(class_weights_raw))
print("Class weights:", class_weights)


Class weights: {0: np.float64(2.5760182584269664), 1: np.float64(3.006762295081967), 2: np.float64(1.7534655831739963), 3: np.float64(0.81227856510186), 4: np.float64(1.1979915088177662), 5: np.float64(0.43773866348448687), 6: np.float64(0.47838419405320814), 7: np.float64(3.7278963414634148)}


In [7]:
model.compile(
    optimizer=RMSprop(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [8]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint(filepath=os.path.join(output_dir, 'best_model.h5'), monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [9]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS,
    class_weight=class_weights,
    callbacks=callbacks
)

C:\Users\manas\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
  45/1835 ━━━━━━━━━━━━━━━━━━━━ 8:02 270ms/step - accuracy: 0.8243 - loss: 0.8599

KeyboardInterrupt: 

In [ ]:
h5_path = os.path.join(output_dir, "focused_final.h5")
keras_path = os.path.join(output_dir, "focused_final.keras")
tflite_path = os.path.join(output_dir, "focused_final.tflite")

model.save(h5_path)
model.save(keras_path)
print(f"Saved: {h5_path} and {keras_path}")
